In [ ]:
import sys

import boto3
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters

In [6]:
# constants
MY_TIMEZONE='UTC' # update to your local timezone or use 'UTC'
DATETIME_FORMAT='%y-%m-%d %H:%M' # update to your local date/time format

In [7]:
def parse(x):
    x = pd.to_datetime(x, infer_datetime_format=True, unit='s', utc=True)
    x = x.tz_convert(MY_TIMEZONE)
    return x

In [11]:
%%time

client = boto3.client('iotanalytics')
nrows_read = None # specify 'None' if want to read whole data set (10k records)
dataset = 'iot_vodafone_dataset'
data_location = client.get_dataset_content(datasetName=dataset)['entries'][0]['dataURI']

#date_parser=parse,

df = pd.read_csv(data_location,
                 nrows=nrows_read,
                 header=0,
                 low_memory=False,
                 infer_datetime_format=True,
                 date_parser=parse)

CPU times: user 31.5 ms, sys: 3.92 ms, total: 35.4 ms
Wall time: 259 ms


In [14]:
# clean up the DataFrame
#df = df.drop(columns='__dt')
df.columns
#df = df.sort_values(by='ts', ascending=True)

Index(['id', 'id_wasp', 'id_secret', 'sensor', 'value', 'datetime'], dtype='object')

In [ ]:
# transform metadata attribute, from device registry, to valid json object
#df['metadata'].replace('{','{"',regex=True, inplace = True)
#df['metadata'].replace('=','":"',regex=True, inplace = True)
#df['metadata'].replace(', ','","',regex=True, inplace = True)
#df['metadata'].replace(':"{"',':{"',regex=True, inplace = True)
#df['metadata'].replace('}",','},',regex=True, inplace = True)
#df['metadata'].replace('}','"}',regex=True, inplace = True)

In [18]:
df.tail()

,id,id_wasp,id_secret,sensor,value,datetime
995,1021359,STATION_01,22626F057C1054B4,PLV3,0.00,2021-07-28 05:50:27
996,1021355,STATION_01,22626F057C1054B4,SOIL3,4098.36,2021-07-28 05:50:17
997,1021356,STATION_01,22626F057C1054B4,BAT,95.00,2021-07-28 05:50:17
998,1021354,STATION_01,22626F057C1054B4,SOIL2,4132.23,2021-07-28 05:50:17
999,1021353,STATION_01,22626F057C1054B4,SOIL1,4761.90,2021-07-28 05:50:17


In [ ]:

# filter temp/humidity outliers (>1% & <99%)
df = df.loc[df['temp'] > df.groupby('device').temp.transform(lambda x: x.quantile(.01))]
df = df.loc[df['temp'] < df.groupby('device').temp.transform(lambda x: x.quantile(.99))]

df = df.loc[df['humidity'] > df.groupby('device').humidity.transform(lambda x: x.quantile(.01))]
df = df.loc[df['humidity'] < df.groupby('device').humidity.transform(lambda x: x.quantile(.99))]

In [ ]:
# group by device
groups = df.groupby('device')

In [ ]:
print('DataFrame Stats')
print('-------------')
print('Record count: {:,}'.format(df['temp'].count()))
print('DataFrame size (MB): {:,.2f}'.format(sys.getsizeof(df)/1024/1024))
print('-------------')
print('Time range (min): {:%Y-%m-%d %H:%M:%S %Z}'.format(df.index[1]))
print('Time range (max): {:%Y-%m-%d %H:%M:%S %Z}'.format(df.index[-1]))
print('Temperature (min): {:.2f}'.format(df['temp'].min()))
print('Temperature (max): {:.2f}'.format(df['temp'].max()))
print('Humidity (min): {:.2f}{}'.format(df['humidity'].min(), '%'))
print('Humidity (max): {:.2f}{}'.format(df['humidity'].max(), '%'))
print('-------------')
print('Record count:\n{}'.format(groups.size()))
# print('Temperature (min):\n{}'.format(groups['temp'].min()))
# print('Temperature (max):\n{}'.format(groups['temp'].max()))
# print('Humidity (min)\n{}'.format(groups['humidity'].min()))
# print('Humidity (max):\n{}'.format(groups['humidity'].max()))

In [ ]:
# example of extracting value from metadata attribute
metadata = pd.read_json(df['metadata'][0], orient='records', typ='series')
print(metadata['thingname'])

In [ ]:
# matplotlib datetime config
plt.rcParams['timezone']=MY_TIMEZONE
register_matplotlib_converters()
myFmt = DateFormatter(DATETIME_FORMAT)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(18, 9))
for device, group in groups:
    ax.plot(group.temp,
            group.humidity,
            marker='o',
            linestyle='',
            alpha=.5,
            ms=10,
            label=device)
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Temperature vs. Humidity')
plt.xlabel('Temperature (˚F)')
plt.ylabel('Humidity (%)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 9))
for device, group in groups:
    group.mean = group.temp.rolling(window=20).mean()
    ax.plot(group.mean,
            label=device)
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(myFmt)
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Temperature Comparison over Time')
plt.ylabel('Temperature (˚F)')
plt.xlabel('Time')
plt.show()

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(18, 9))
for device, group in groups:
    group.mean = group.humidity.rolling(window=20).mean()
    ax.plot(group.mean,
            label=device)
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(myFmt)
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Humidity Comparison over Time')
plt.ylabel('Humidity (%)')
plt.xlabel('Time')
plt.show()